#### Importing All Necessary Libraries

In [58]:
import os
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

#### Data Ingestion

*Loading All Text Documents*

In [19]:
directory_path = 'c:/Users/Admin/Desktop/RAG Project/data'

docs = []

for filename in os.listdir(directory_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(directory_path, filename)
        print(f"Loading file: {filename}")  # Print the name of each file being loaded
        loader = UnstructuredPDFLoader(file_path = file_path)
        data = loader.load()
        docs.extend(data)  # Append the data from each file to the 'docs' list

Loading file: 2012.pdf
Loading file: 2023.pdf
Loading file: alternative_med.pdf
Loading file: DRAP-2012-_As-Amended-till-Feb-2022.pdf
Loading file: DRAP-Regulatory-Fee-30.08.2024.pdf
Loading file: Drug-Pricing-Policy-2018-Amended-upto-12.07.2023-2.pdf
Loading file: DrugRules.pdf
Loading file: DRUGS (LICENSING, REGISTERING AND ADVERTISING) .pdf
Loading file: Drugs-Import-Export-Rules-1976.pdf
Loading file: DrugsFederalInspectorsLabs.pdf
Loading file: drugspecificationrules.pdf
Loading file: ethical marketin.pdf
Loading file: GMPInspectionCommitteeAppealRegulations2017.pdf
Loading file: guidelines.pdf
Loading file: labelling.pdf
Loading file: penaties and offences.pdf
Loading file: Punjab-Drug-Sale-Rules.pdf
Loading file: Schedules of medicine.pdf
Loading file: The-Drugs-_Research_-Rules-1978.pdf


*Chunking*

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2800, chunk_overlap = 300)
chunks = text_splitter.split_documents(docs)

In [21]:
len(chunks)

1334

*Vector Embeddings*

In [ ]:
persist_directory = 'chroma/drap-embeddings'
embedding_function = OllamaEmbeddings(model = "nomic-embed-text", show_progress = True)

In [35]:
vector_db = Chroma.from_documents(
    documents = chunks, 
    embedding = embedding_function,
    collection_name = "drap-embeddings",
    persist_directory = persist_directory
)

OllamaEmbeddings: 100%|██████████| 1334/1334 [4:29:13<00:00, 12.11s/it]  


In [36]:
vector_db.persist()

C:\Users\Admin\AppData\Local\Temp\ipykernel_14992\2766552738.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [37]:
print(vector_db._collection.count())

1334


#### Retrieval

*LLM - LLAMA 3.2 (from Ollama)*

In [38]:
local_model = "llama3.2"
llm = ChatOllama(model = local_model)

*Building a Retriever (Chroma Retriever)*

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables = ["question"],
    template = """
    Generate 2 different versions of the given user question to retrieve relevant documents from a vector database.
    Provide these alternative questions separated by newlines.
    
    Original question: {question}
    """,
)

In [68]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt = QUERY_PROMPT
)

# RAG prompt
template = """
The user has queries regarding the Drug Regulatory Authority regulations and guidelines. You are an AI assistant who is responsible for answering any questions the user might have from a legal perspective or just for knowledge purposes too.
If they ask for advice, then give advice. If they ask for a list of things, then provide a list. If they want to know how certain things are done then understand the context and answer appropriately.
Answer the question professionally and as efficiently as possible based ONLY on the following context and give a reference from the document you cite your answer from:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Generation

*Building a Chained Pipeline*

In [70]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [43]:
def chat(question):
    response = chain.invoke(question)
    return response

#### Example Question-Answering

*Question 1*

In [71]:
chat("What are the legal consequences for obstructing an officer during an investigation?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.93s/it]


"The text does not explicitly state the specific legal consequences for obstructing an officer during an investigation. However, based on the information provided in the document (Section 29 of The Drugs Act 1976), it can be inferred that obstruction of an inspector or other authority may result in punishment with imprisonment for a term not exceeding one year, or with fine not exceeding one lakh rupees, or both.\n\nHowever, to provide more specific guidance, I'll need to refer to the Code of Criminal Procedure, 1898 (Act V of 1898), which mentions that an offence punishable under Schedule-III other than an offence mentioned in clause (1) of that Schedule shall be non-cognizable.\n\nIt's also worth noting that the penalties and consequences mentioned in Section 29 are for contraventions under The Drugs Act 1976, and not specifically for obstructing an officer during an investigation. Therefore, it would be best to consult with a legal professional or a relevant authority for more speci

The text does not explicitly state the specific legal consequences for obstructing an officer during an investigation. However, based on the information provided in the document (Section 29 of The Drugs Act 1976), it can be inferred that obstruction of an inspector or other authority may result in punishment with imprisonment for a term not exceeding one year, or with fine not exceeding one lakh rupees, or both.

However, to provide more specific guidance, I'll need to refer to the Code of Criminal Procedure, 1898 (Act V of 1898), which mentions that an offence punishable under Schedule-III other than an offence mentioned in clause (1) of that Schedule shall be non-cognizable.

It's also worth noting that the penalties and consequences mentioned in Section 29 are for contraventions under The Drugs Act 1976, and not specifically for obstructing an officer during an investigation. Therefore, it would be best to consult with a legal professional or a relevant authority for more specific guidance on this matter.

*Question 2*

In [72]:
chat("What is the penalty for exporting, importing, manufacturing, or selling spurious or unregistered drugs?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.43s/it]


'The penalty for exporting, importing, manufacturing, or selling spurious or unregistered drugs in Pakistan can be taken under the Drug Regulatory Authority of Pakistan Act 2012.\n\nAs per the act, there are severe penalties and legal consequences for those involved in such activities. These include:\n\n*   Imprisonment \n*   Fine\n*   Seizure of goods'

The penalty for exporting, importing, manufacturing, or selling spurious or unregistered drugs in Pakistan can be taken under the Drug Regulatory Authority of Pakistan Act 2012.

As per the act, there are severe penalties and legal consequences for those involved in such activities. These include:
*   Imprisonment
*   Fine
*   Seizure of goods

*Question 3*

In [73]:
chat("Which drugs are restricted to sale only with a prescription from a registered medical practitioner?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.46s/it]


'According to the text, there is no specific list of drugs that are explicitly mentioned as being restricted to sale only with a prescription from a registered medical practitioner.\n\nHowever, it can be inferred that "special medicinal products" such as:\n\n* Dietary supplements\n* Nutraceuticals\n* Cosmeceuticals\n* Contrast/dyes\n* Radiopharmaceuticals\n* Surgical devices\n* Implants\n* Alternative therapies\n\nare regulated by specific policies and procedures, and their use may require a prescription or authorization from a registered medical practitioner.\n\nAdditionally, the text mentions that there are "policies and procedures" for the procurement, control, and use of these special medicinal products, which suggests that they may be subject to certain restrictions or requirements.'

According to the text, there is no specific list of drugs that are explicitly mentioned as being restricted to sale only with a prescription from a registered medical practitioner.

However, it can be inferred that "special medicinal products" such as:

* Dietary supplements
* Nutraceuticals\n* Cosmeceuticals
* Contrast/dyes
* Radiopharmaceuticals
* Surgical devices
* Implants
* Alternative therapies are regulated by specific policies and procedures, and their use may require a prescription or authorization from a registered medical practitioner.

Additionally, the text mentions that there are "policies and procedures" for the procurement, control, and use of these special medicinal products, which suggests that they may be subject to certain restrictions or requirements.

*Question 4*

In [74]:
chat("What fine is imposed for selling a drug labeled as “Physician Sample – Not for Sale”?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


'The question does not mention anything about imposing a fine specifically for labeling a drug with the words "Physician Sample - Not for Sale". However, I can provide some general information.\n\nAccording to Rule 6(a) of the Punjab-Drug-Sale-Rules, any person who sells or offers to sell without a licence is punishable with imprisonment for a term which may extend to three years and fine which may extend to twenty-five thousand rupees.'

The question does not mention anything about imposing a fine specifically for labeling a drug with the words "Physician Sample - Not for Sale". However, I can provide some general information.

According to Rule 6(a) of the Punjab-Drug-Sale-Rules, any person who sells or offers to sell without a licence is punishable with imprisonment for a term which may extend to three years and fine which may extend to twenty-five thousand rupees.

*Question 5*

In [75]:
chat("What are the prerequisites for obtaining a license to open and operate a pharmacy or medical store?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.51s/it]


'According to the text, the prerequisites for obtaining a license to open and operate a pharmacy or medical store in Pakistan are:\n\n1. The applicant must submit an application in Form 8(A) or Form 8(B) along with the required fee.\n2. The fee for a pharmacy license is Rs. 3,000, and for a medical store license, it is Rs. 2,000 (for new applications).\n3. For renewal of licenses:\n\t* Pharmacy license: Rs. 1,000\n\t* Medical store license: Rs. 1,000\n4. The applicant must pay 50% of the fee for changing the qualified person or obtaining a duplicate copy of the license.\n5. The licensing authority may refuse an application if it is not expedient in public interest to grant a license.\n\nAdditionally, the following conditions must be met:\n\n* The applicant must meet the storage conditions and store the drugs properly (for medical stores).\n* If the applicant intends to sell or distribute drugs at more than one place, they must apply for separate licenses for each location.'

According to the text, the prerequisites for obtaining a license to open and operate a pharmacy or medical store in Pakistan are:

1. The applicant must submit an application in Form 8(A) or Form 8(B) along with the required fee.
2. The fee for a pharmacy license is Rs. 3,000, and for a medical store license, it is Rs. 2,000 (for new applications).
3. For renewal of licenses:

* Pharmacy license: Rs. 1,000
* Medical store license: Rs. 1,000

4. The applicant must pay 50% of the fee for changing the qualified person or obtaining a duplicate copy of the license.
5. The licensing authority may refuse an application if it is not expedient in public interest to grant a license

Additionally, the following conditions must be met:

* The applicant must meet the storage conditions and store the drugs properly (for medical stores).

If the applicant intends to sell or distribute drugs at more than one place, they must apply for separate licenses for each location.

#### Experimenting With Different Retriever Methods

**BM25 Retriever**

In [76]:
texts = [doc.page_content for doc in chunks]

bm25 = BM25Retriever.from_texts(texts)

bm25_retriever = MultiQueryRetriever.from_llm(
    bm25,
    llm,
    prompt = QUERY_PROMPT
)

chain = (
    {"context": bm25_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

*Question 1*

In [77]:
chat("What are the legal consequences for obstructing an officer during an investigation?")

"The provided text does not mention anything about obstructing an officer during an investigation or its related legal consequences. It appears to be a document providing guidelines for reporting Adverse Events (AEs) and Adverse Reactions to Medicinal Products (ARMPs) through a mobile application, specifically Med Safety.\n\nHowever, if you're looking for general information on the legal consequences of obstructing an officer during an investigation, it can vary depending on the jurisdiction and specific circumstances. In many places, obstructing an officer or interfering with their duties can be considered a crime, such as:\n\n1. Obstruction of Justice: This is a federal offense in some countries that carries penalties such as fines, imprisonment, or both.\n2. Interference with Law Enforcement: Some jurisdictions consider this a crime, punishable by law, and may result in fines or prison time.\n3. Resisting Arrest: If an individual resists arrest or attempts to flee from the scene, th

The provided text does not mention anything about obstructing an officer during an investigation or its related legal consequences. It appears to be a document providing guidelines for reporting Adverse Events (AEs) and Adverse Reactions to Medicinal Products (ARMPs) through a mobile application, specifically Med Safety.

However, if you're looking for general information on the legal consequences of obstructing an officer during an investigation, it can vary depending on the jurisdiction and specific circumstances. In many places, obstructing an officer or interfering with their duties can be considered a crime, such as:

1. Obstruction of Justice: This is a federal offense in some countries that carries penalties such as fines, imprisonment, or both.
2. Interference with Law Enforcement: Some jurisdictions consider this a crime, punishable by law, and may result in fines or prison time.\n3. Resisting Arrest: If an individual resists arrest or attempts to flee from the scene, they may be charged with resisting arrest.

The specific penalties can range widely depending on the jurisdiction, circumstances, and the severity of the offense.

*Question 2*

In [78]:
chat("What is the penalty for exporting, importing, manufacturing, or selling spurious or unregistered drugs?")

"According to the provided text snippet, there is no explicit mention of a specific penalty for exporting, importing, manufacturing, or selling spurious or unregistered drugs. However, the text does mention that the Authority may engage in regulating, enforcing, and monitoring advertisement rules and banning false advertisements.\n\nIn general, the regulations regarding the sale and distribution of pharmaceuticals, including penalties for spurious or unregistered drugs, are typically outlined by government agencies such as the Food and Drug Administration (FDA) or equivalent bodies in other countries. These regulations can vary depending on the jurisdiction, but they often include provisions for severe penalties for those involved in the unauthorized manufacture, sale, or distribution of pharmaceuticals.\n\nSome possible penalties for exporting, importing, manufacturing, or selling spurious or unregistered drugs could include:\n\n*   Fines and monetary penalties\n*   Imprisonment or im

According to the provided text snippet, there is no explicit mention of a specific penalty for exporting, importing, manufacturing, or selling spurious or unregistered drugs. However, the text does mention that the Authority may engage in regulating, enforcing, and monitoring advertisement rules and banning false advertisements.

In general, the regulations regarding the sale and distribution of pharmaceuticals, including penalties for spurious or unregistered drugs, are typically outlined by government agencies such as the Food and Drug Administration (FDA) or equivalent bodies in other countries. These regulations can vary depending on the jurisdiction, but they often include provisions for severe penalties for those involved in the unauthorized manufacture, sale, or distribution of pharmaceuticals.

Some possible penalties for exporting, importing, manufacturing, or selling spurious or unregistered drugs could include:

*   Fines and monetary penalties
*   Imprisonment or imprisonment terms
*   Revocation of licenses to manufacture or sell pharmaceuticals
*   Forfeiture of assets or profits derived from the sale of spurious or unregistered drugs

It's essential to consult specific laws, regulations, or guidelines in a particular country or region for accurate information on penalties related to the sale or distribution of spurious or unregistered drugs.

*Question 3*

In [79]:
chat("Which drugs are restricted to sale only with a prescription from a registered medical practitioner?")

'The question is not explicitly stated in the text, but based on the context of the Adverse Events Reporting Guidelines for Healthcare Professionals, it can be inferred that the answer refers to therapeutic goods.\n\nAccording to the guidelines, there is no specific mention of "drugs" being restricted to sale only with a prescription from a registered medical practitioner. However, section 7.1.2 mentions that products which are classified as Class A or Class B under the Drugs and Medicines Ordinance (DMO) should not be sold without a valid prescription.\n\nIn Pakistan, Class A and Class B drugs are typically those that require a prescription for sale due to their potential for abuse, dependency, or other safety concerns. Examples of such drugs might include controlled substances, such as opioids or sedatives, as well as certain medications used to treat psychiatric disorders or addictive diseases.\n\nExamples of such restricted drugs in Pakistan would include:\n\n* Opioids (e.g., morph

The question is not explicitly stated in the text, but based on the context of the Adverse Events Reporting Guidelines for Healthcare Professionals, it can be inferred that the answer refers to therapeutic goods.

According to the guidelines, there is no specific mention of "drugs" being restricted to sale only with a prescription from a registered medical practitioner. However, section 7.1.2 mentions that products which are classified as Class A or Class B under the Drugs and Medicines Ordinance (DMO) should not be sold without a valid prescription.

In Pakistan, Class A and Class B drugs are typically those that require a prescription for sale due to their potential for abuse, dependency, or other safety concerns. Examples of such drugs might include controlled substances, such as opioids or sedatives, as well as certain medications used to treat psychiatric disorders or addictive diseases.

Examples of such restricted drugs in Pakistan would include:

* Opioids (e.g., morphine, oxycodone)* Benzodiazepines (e.g., diazepam, alprazolam)
* Stimulants (e.g., amphetamines, methylphenidate)
* Certain psychiatric medications (e.g., risperidone, aripiprazole)

Please note that this is not an exhaustive list, and the specific drugs restricted to sale only with a prescription from a registered medical practitioner may vary depending on local regulations and guidelines.

*Question 4*

In [80]:
chat("What fine is imposed for selling a drug labeled as “Physician Sample – Not for Sale”?")

'The text does not provide information on the fine imposed for selling a drug labeled as "Physician Sample – Not for Sale". However, I can tell you that according to the DRA Guidelines, selling a drug labeled as "Physician Sample – Not for Sale" is considered an offense and is punishable under the law. The exact penalty or fine imposed would depend on the specific circumstances and the laws in place at the time of the incident.'

The text does not provide information on the fine imposed for selling a drug labeled as "Physician Sample – Not for Sale". However, I can tell you that according to the DRA Guidelines, selling a drug labeled as "Physician Sample – Not for Sale" is considered an offense and is punishable under the law. The exact penalty or fine imposed would depend on the specific circumstances and the laws in place at the time of the incident.

*Question 5*

In [81]:
chat("What are the prerequisites for obtaining a license to open and operate a pharmacy or medical store?")

"Unfortunately, this text does not appear to be related to the prerequisites for opening and operating a pharmacy or medical store. The text seems to be a pharmaceutical regulatory guideline document, specifically a set of guidelines for regulatory appeals.\n\nHowever, I can try to provide some general information on the typical prerequisites for obtaining a license to open and operate a pharmacy or medical store.\n\nIn most countries, the prerequisites for opening and operating a pharmacy or medical store typically include:\n\n1. Obtaining a pharmacy license or medical store registration from the relevant authorities.\n2. Completing a training program or apprenticeship in pharmacy or medicine.\n3. Passing a licensing examination to demonstrate proficiency in pharmacy or medical knowledge.\n4. Meeting specific requirements related to education, experience, and character.\n5. Having a valid business license and tax identification number.\n\nPlease note that these prerequisites may vary 

Unfortunately, this text does not appear to be related to the prerequisites for opening and operating a pharmacy or medical store. The text seems to be a pharmaceutical regulatory guideline document, specifically a set of guidelines for regulatory appeals.

However, I can try to provide some general information on the typical prerequisites for obtaining a license to open and operate a pharmacy or medical store.

In most countries, the prerequisites for opening and operating a pharmacy or medical store typically include:

1. Obtaining a pharmacy license or medical store registration from the relevant authorities.
2. Completing a training program or apprenticeship in pharmacy or medicine.
3. Passing a licensing examination to demonstrate proficiency in pharmacy or medical knowledge.
4. Meeting specific requirements related to education, experience, and character.
5. Having a valid business license and tax identification number.

Please note that these prerequisites may vary depending on the country, state, or region where you plan to open your pharmacy or medical store. It's essential to check with the relevant authorities for specific requirements and regulations.

**Ensemble Retriever**

In [82]:
ensemble_retriever = EnsembleRetriever(
    retrievers = [bm25_retriever, retriever], weights = [0.5, 0.5]
)

chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

*Question 1*

In [83]:
chat("What are the legal consequences for obstructing an officer during an investigation?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.42s/it]


"The text provided does not specifically mention the legal consequences for obstructing an officer during an investigation. However, I can provide general information on this topic.\n\nObstructing an officer during an investigation is considered a serious offense in many jurisdictions and can result in various legal consequences. The specific penalties may vary depending on the jurisdiction, the severity of the obstruction, and other factors.\n\nIn some cases, obstructing an officer during an investigation may be charged as a misdemeanor or felony offense, and can carry penalties such as:\n\n*   Fines\n*   Imprisonment (including short-term or extended sentences)\n*   Probation\n*   Community service\n\nAdditionally, in some jurisdictions, obstructing an officer during an investigation may also result in additional consequences, such as:\n\n*   Suspension of driver's license\n*   Revocation of professional licenses\n*   Mandatory attendance at a court-ordered anger management program o

The text provided does not specifically mention the legal consequences for obstructing an officer during an investigation. However, I can provide general information on this topic.

Obstructing an officer during an investigation is considered a serious offense in many jurisdictions and can result in various legal consequences. The specific penalties may vary depending on the jurisdiction, the severity of the obstruction, and other factors.

In some cases, obstructing an officer during an investigation may be charged as a misdemeanor or felony offense, and can carry penalties such as:
*   Fines
*   Imprisonment (including short-term or extended sentences)
*   Probation
*   Community service

Additionally, in some jurisdictions, obstructing an officer during an investigation may also result in additional consequences, such as:

*   Suspension of driver's license
*   Revocation of professional licenses
*   Mandatory attendance at a court-ordered anger management program or other treatment programs
*   Restitution to the victim(s)

It is essential to note that these are general examples and the specific penalties may vary depending on the jurisdiction, the severity of the obstruction, and other factors.

If you have concerns about obstructing an officer during an investigation, I recommend consulting with a qualified attorney or law enforcement professional for guidance.

*Question 2*

In [84]:
chat("What is the penalty for exporting, importing, manufacturing, or selling spurious or unregistered drugs?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.45s/it]


'The text does not specifically mention a penalty for exporting, importing, manufacturing, or selling spurious or unregistered drugs. However, it mentions that "each company shall provide a detailed summary of all expenditures incurred on institutions or healthcare professionals on account of marketing, honoraria, travel, subsistence expenses, grants and any other related financial transaction" to the relevant tax authorities and the Authority.\n\nIt is likely that exporting, importing, manufacturing, or selling spurious or unregistered drugs would be considered a breach of the rules and could result in penalties, but the specific penalty is not mentioned in this text.'

The text does not specifically mention a penalty for exporting, importing, manufacturing, or selling spurious or unregistered drugs. However, it mentions that "each company shall provide a detailed summary of all expenditures incurred on institutions or healthcare professionals on account of marketing, honoraria, travel, subsistence expenses, grants and any other related financial transaction" to the relevant tax authorities and the Authority.

It is likely that exporting, importing, manufacturing, or selling spurious or unregistered drugs would be considered a breach of the rules and could result in penalties, but the specific penalty is not mentioned in this text.

*Question 3*

In [85]:
chat("Which drugs are restricted to sale only with a prescription from a registered medical practitioner?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.41s/it]


'According to the provided text, High Alert Medication is restricted to sale only with a prescription from a registered medical practitioner.'

According to the provided text, High Alert Medication is restricted to sale only with a prescription from a registered medical practitioner.

*Question 4*

In [86]:
chat("What fine is imposed for selling a drug labeled as “Physician Sample – Not for Sale”?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.42s/it]


'The question does not mention the imposition of a fine, but rather asks about the label on a sample of a drug. According to the rules, a sample of a drug may be provided to physicians or dentists or pharmacists or veterinarians or medical institutions in reasonable quantities and in reduced packings marked with the words "Physician Sample - Not for Sale".'

The question does not mention the imposition of a fine, but rather asks about the label on a sample of a drug. According to the rules, a sample of a drug may be provided to physicians or dentists or pharmacists or veterinarians or medical institutions in reasonable quantities and in reduced packings marked with the words "Physician Sample - Not for Sale".

*Question 5*

In [87]:
chat("What are the prerequisites for obtaining a license to open and operate a pharmacy or medical store?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.35s/it]


'According to the rules mentioned in the text, the prerequisites for obtaining a license to open and operate a pharmacy or medical store include:\n\n1. The pharmacy or medical store must be registered with the relevant authorities.\n2. The premises where the pharmacy or medical store is located must meet certain requirements, such as having proper infrastructure, facilities, and working conditions.\n3. The licensee (the person or entity applying for the license) must have a valid prescription from a licensed medical practitioner.\n4. The licensee must provide proof of qualifications, experience, and character to the licensing authority.\n5. The pharmacy or medical store must comply with all relevant laws, regulations, and rules governing the sale and dispensing of pharmaceuticals and other medicinal products.\n\nAdditionally, certain documents such as an invoice and warranty shall bear the full name and address of the purchaser and shall be signed by the warrantor clearly indicating hi

According to the rules mentioned in the text, the prerequisites for obtaining a license to open and operate a pharmacy or medical store include:

1. The pharmacy or medical store must be registered with the relevant authorities.
2. The premises where the pharmacy or medical store is located must meet certain requirements, such as having proper infrastructure, facilities, and working conditions.
3. The licensee (the person or entity applying for the license) must have a valid prescription from a licensed medical practitioner.
4. The licensee must provide proof of qualifications, experience, and character to the licensing authority.
5. The pharmacy or medical store must comply with all relevant laws, regulations, and rules governing the sale and dispensing of pharmaceuticals and other medicinal products.

Additionally, certain documents such as an invoice and warranty shall bear the full name and address of the purchaser and shall be signed by the warrantor clearly indicating his name and shall be dated. The manufacturer, importer or seller of a drug must maintain record of purchase or sale of a drug and preserve the record for at least three years containing the required particulars.

It is also mentioned that certain substances specified in Schedule E and listed poisons should be stored in a specific manner to prevent leakage arising from ordinary risks of handling and transport.